In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV

In [ ]:
# Ruta al archivo Parquet
ruta_archivo_parquet = r'C:\Users\guard\OneDrive\Desktop\Henry Data Science\Proyecto-FInal\Datos\gm_rev_NJNY_202502182030.parquet'

# Leer el archivo Parquet en un DataFrame
df = pd.read_parquet(ruta_archivo_parquet)

# Asegurarnos de que no hay valores nulos en la columna 'text'
df = df.dropna(subset=['text'])

# Inicializar el analizador de sentimientos VADER
analyzer = SentimentIntensityAnalyzer()

# Definir una función para calcular la puntuación de sentimientos
def analizar_sentimiento(texto):
    if texto is None:
        texto = ""  # Reemplazar valores nulos con cadena vacía
    sentimientos = analyzer.polarity_scores(texto)
    return sentimientos['compound']

# Aplicar la función a la columna 'text' para obtener la puntuación de sentimientos
df['sentimiento_puntuacion'] = df['text'].apply(analizar_sentimiento)


In [4]:
# Crear una columna 'sentimiento' con valores categóricos basado en la puntuación de sentimientos
df['sentimiento'] = df['sentimiento_puntuacion'].apply(lambda x: 'positivo' if x > 0 else 'negativo' if x < 0 else 'neutro')

In [ ]:
# Mostrar la distribución inicial de clases
print("Distribución inicial de clases:", Counter(y_train))

# Aplicar SMOTE para realizar el sobremuestreo
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Mostrar la nueva distribución de clases
print("Distribución después de SMOTE:", Counter(y_train_res))


Distribución inicial de clases: Counter({'positivo': 61102, 'negativo': 6636, 'neutro': 4862})
Distribución después de SMOTE: Counter({'positivo': 61102, 'negativo': 61102, 'neutro': 61102})


In [ ]:
# Definir los parámetros de la cuadrícula
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],  # Normas de penalización L1 y L2
    'solver': ['liblinear']  # Solvers compatibles con L1 y L2
}

# Inicializar el modelo
modelo = LogisticRegression()

# Configurar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)

# Entrenar el modelo con la búsqueda en cuadrícula
grid_search.fit(X_train_res, y_train_res)

# Obtener los mejores parámetros
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


Mejores parámetros: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


In [9]:
# Entrenar el modelo con los mejores parámetros
modelo_final = LogisticRegression(**best_params)
modelo_final.fit(X_train_res, y_train_res)

# Evaluar el modelo final
y_pred = modelo_final.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    negativo       0.61      0.76      0.68      1633
      neutro       0.72      0.81      0.77      1130
    positivo       0.97      0.94      0.95     15387

    accuracy                           0.91     18150
   macro avg       0.77      0.84      0.80     18150
weighted avg       0.92      0.91      0.92     18150



In [10]:
# Guardar el modelo final y el vectorizador
joblib.dump(modelo_final, 'modelo_sentimientos_final.pkl')
joblib.dump(vectorizer, 'vectorizador_tfidf.pkl')


['vectorizador_tfidf.pkl']